# Build a Simple Retrieval-Augmented Generation (RAG) Pipeline

In this use case, we show how to build and evaluate a simple RAG pipeline with LightRAG. RAG (Retrieval-Augmented Generation) pipelines leverage a retriever to fetch relevant context from a knowledge base (e.g., a document database) which is then fed to an LLM generator with the query to produce the answer. This allows the model to generate more contextually relevant answers.

In [1]:
# Import needed modules, including modules for loading datasets, constructing a RAG pipeline, and evaluating the performance of the RAG pipeline.
import yaml
from typing import Any, List, Optional, Union

from datasets import load_dataset

from lightrag.core.types import Document
from lightrag.core.component import Component, Sequential
from lightrag.core.embedder import Embedder
from lightrag.core.document_splitter import DocumentSplitter
from lightrag.core.data_components import (
    RetrieverOutputToContextStr,
    ToEmbeddings,
)
from lightrag.components.retriever import FAISSRetriever
from lightrag.core.generator import Generator
from lightrag.core.db import LocalDocumentDB
from lightrag.core.string_parser import JsonParser

In [2]:
# Here, we use the OpenAIClient in the Generator as an example, but you can use any other clients (with the corresponding API Key as needed)
from lightrag.components.model_client import OpenAIClient
OPENAI_API_KEY="YOUR_API_KEY" # Replace with your OpenAI API Key, or you can put it in a .env file

**Define the configuration for the RAG pipeline**. We load the configuration from a YAML file. This configuration specifies the components of the RAG pipeline, including the text_splitter, vectorizer, retriever, and generator.

In [3]:
# Define the configuration settings for the RAG pipeline.
with open("./simple_rag.yaml", "r") as file:
    settings = yaml.safe_load(file)
print(settings)

{'vectorizer': {'batch_size': 100, 'model_kwargs': {'model': 'text-embedding-3-small', 'dimensions': 256, 'encoding_format': 'float'}}, 'retriever': {'top_k': 2}, 'generator': {'model': 'gpt-3.5-turbo', 'temperature': 0.3, 'stream': False}, 'text_splitter': {'split_by': 'sentence', 'chunk_size': 1, 'chunk_overlap': 0}}


**Load a dataset**. Here, We use the [HotpotQA](https://huggingface.co/datasets/hotpotqa/hotpot_qa) dataset as an example. Each data sample in HotpotQA has *question*, *answer*, *context* and *supporting_facts* selected from the whole context.

In [4]:
# Load the HotpotQA dataset. We select a subset of the dataset for demonstration purposes.
dataset = load_dataset(path="hotpot_qa", name="fullwiki")
dataset = dataset["train"].select(range(5))
print(f"example: {dataset[0]}")
print(f"ground truth context: {dataset[0]['supporting_facts']}")

example: {'id': '5a7a06935542990198eaf050', 'question': "Which magazine was started first Arthur's Magazine or First for Women?", 'answer': "Arthur's Magazine", 'type': 'comparison', 'level': 'medium', 'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'], 'sent_id': [0, 0]}, 'context': {'title': ['Radio City (Indian radio station)', 'History of Albanian football', 'Echosmith', "Women's colleges in the Southern United States", 'First Arthur County Courthouse and Jail', "Arthur's Magazine", '2014–15 Ukrainian Hockey Championship', 'First for Women', 'Freeway Complex Fire', 'William Rast'], 'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.", ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).', ' It plays Hindi, English and regional songs.', ' It was launched in Hyderabad in March 2006, in Chenna

/Users/mengliu/Library/Caches/pypoetry/virtualenvs/lightrag-project-OrKUABKc-py3.12/lib/python3.12/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


**Define a simple RAG pipeline**. Define a RAG pipeline by specifying the key components, such as *vectorizer*, *retriever*, and *generator*. For more information on these components, refer to the developer notes.

In [7]:
# The defined RAG pipeline.
class RAG(Component):

    def __init__(self, settings: dict):
        super().__init__()
        self.vectorizer_settings = settings["vectorizer"]
        self.retriever_settings = settings["retriever"]
        self.generator_model_kwargs = settings["generator"]
        self.text_splitter_settings = settings["text_splitter"]

        vectorizer = Embedder(
            model_client=OpenAIClient(),
            model_kwargs=self.vectorizer_settings["model_kwargs"],
        )

        text_splitter = DocumentSplitter(
            split_by=self.text_splitter_settings["split_by"],
            split_length=self.text_splitter_settings["chunk_size"],
            split_overlap=self.text_splitter_settings["chunk_overlap"],
        )
        self.data_transformer = Sequential(
            text_splitter,
            ToEmbeddings(
                vectorizer=vectorizer,
                batch_size=self.vectorizer_settings["batch_size"],
            ),
        )
        self.data_transformer_key = self.data_transformer._get_name()
        # initialize retriever, which depends on the vectorizer too
        self.retriever = FAISSRetriever(
            top_k=self.retriever_settings["top_k"],
            dimensions=self.vectorizer_settings["model_kwargs"]["dimensions"],
            vectorizer=vectorizer,
        )
        self.retriever_output_processors = RetrieverOutputToContextStr(deduplicate=True)

        self.db = LocalDocumentDB()

        # initialize generator
        self.generator = Generator(
            preset_prompt_kwargs={
                "task_desc_str": r"""
                    You are a helpful assistant.

                    Your task is to answer the query that may or may not come with context information.
                    When context is provided, you should stick to the context and less on your prior knowledge to answer the query.

                    Output JSON format:
                    {
                        "answer": "The answer to the query",
                    }"""
            },
            model_client=OpenAIClient(),
            model_kwargs=self.generator_model_kwargs,
            output_processors=JsonParser(),
        )
        self.tracking = {"vectorizer": {"num_calls": 0, "num_tokens": 0}}

    def build_index(self, documents: List[Document]):
        self.db.load_documents(documents)
        self.map_key = self.db.map_data()
        print(f"map_key: {self.map_key}")
        self.data_key = self.db.transform_data(self.data_transformer)
        print(f"data_key: {self.data_key}")
        self.transformed_documents = self.db.get_transformed_data(self.data_key)
        self.retriever.build_index_from_documents(self.transformed_documents)

    def generate(self, query: str, context: Optional[str] = None) -> Any:
        if not self.generator:
            raise ValueError("Generator is not set")

        prompt_kwargs = {
            "context_str": context,
            "input_str": query,
        }
        response = self.generator(prompt_kwargs=prompt_kwargs)
        if response.error:
            raise ValueError(f"Error in generator: {response.error}")
        return response.data

    def call(self, query: str) -> Any:
        retrieved_documents = self.retriever(query)
        # fill in the document
        for i, retriever_output in enumerate(retrieved_documents):
            retrieved_documents[i].documents = [
                self.transformed_documents[doc_index]
                for doc_index in retriever_output.doc_indexes
            ]
        # convert all the documents to context string
        context_str = self.retriever_output_processors(retrieved_documents)

        return self.generate(query, context=context_str), context_str

To run the RAG piepline for each example in the dataset, we need to first **build the index** and then **call the pipeline**. For each sample in the dataset, we create a list of documents to retrieve from according to its corresponding *context* in the dataset. Each document has a title and a list of sentences. We use the `Document` class from `lightrag.core.types` to represent each document.

In [ ]:
# To get the ground truth context string from the supporting_facts filed in HotpotQA. This function is specific to the HotpotQA dataset.
def get_supporting_sentences(
    supporting_facts: dict[str, list[Union[str, int]]], context: dict[str, list[str]]
) -> List[str]:
    """
    Extract the supporting sentences from the context based on the supporting facts.
    """
    extracted_sentences = []
    for title, sent_id in zip(supporting_facts["title"], supporting_facts["sent_id"]):
        if title in context["title"]:
            index = context["title"].index(title)
            sentence = context["sentences"][index][sent_id]
            extracted_sentences.append(sentence)
    return extracted_sentences


all_questions = []
all_retrieved_context = []
all_gt_context = []
all_pred_answer = []
all_gt_answer = []
for data in dataset:
    # build the document list
    num_docs = len(data["context"]["title"])
    doc_list = [
        Document(
            meta_data={"title": data["context"]["title"][i]},
            text=" ".join(data["context"]["sentences"][i]),
        )
        for i in range(num_docs)
    ]
    # rag = RAG(settings)
    # # build the index
    # rag.build_index(doc_list)
    # # call the pipeline
    # query = data["question"]
    # response, context_str = rag.call(query)
    # import ipdb; ipdb.set_trace()
    # gt_context_sentence_list = get_supporting_sentences(
    #     data["supporting_facts"], data["context"]
    # )
    # all_questions.append(query)
    # all_retrieved_context.append(context_str)
    # all_gt_context.append(gt_context_sentence_list)
    # all_pred_answer.append(response["answer"])
    # all_gt_answer.append(data["answer"])
    # print(f"query: {query}")
    # print(f"response: {response['answer']}")
    # print(f"ground truth response: {data['answer']}")
    # print(f"context_str: {context_str}")
    # print(f"ground truth context_str: {gt_context_sentence_list}")


map_key: lightrag.core.db.<lambda>
start to split documents
splitted_doc: [Document(id=54e3ca94-8148-4c1a-8f00-57c3f5c0c074, text=Radio City is India's first private FM radio station and was started on 3 July 2001., meta_data={'title': 'Radio City (Indian radio station)'}, estimated_num_tokens=21, parent_doc_id=a69d50f7-6186-446a-9952-b9975da778ae), Document(id=858e5dba-30ad-4cee-9279-0b481ee14bd7, text=  It broadcasts on 91., meta_data={'title': 'Radio City (Indian radio station)'}, estimated_num_tokens=7, parent_doc_id=a69d50f7-6186-446a-9952-b9975da778ae), Document(id=9aa39091-bdcc-4a7c-9606-3cd816f95d94, text=1 (earlier 91., meta_data={'title': 'Radio City (Indian radio station)'}, estimated_num_tokens=7, parent_doc_id=a69d50f7-6186-446a-9952-b9975da778ae), Document(id=ac16cabd-223c-4f3c-85e0-eb35b83ca922, text=0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003)., meta_data={'title': 'Radio Ci

100%|██████████| 1/1 [00:00<00:00,  1.84it/s]


data_key: Sequential_transformed


: 